# Session 2 - Subqueries


## Definition
- (Always) executed first
- Always between `()`
- Subqueries can be nested at > 1 level.
- Outer Query
    - Inner Query 1
    - Inner Query 2
    - ...

## Example - 1
**What** is the highest salary of all the employees?

In [35]:
USE Xtreme;
SELECT MAX(Salary)​ AS [Highest Salary]
FROM Employee;

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.079

class,unitprice,(No column name)
NULL,NULL,NULL
1,4.59,3301
2,78.91,403000


**Who** has the highest salary?

In [34]:
USE Xtreme;
SELECT 
 Lastname
,Firstname
,Salary​
FROM Employee​
WHERE Salary = (SELECT MAX(Salary)​ FROM Employee)

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.087

Lastname,Firstname,Salary
Fuller,Andrew,90000.00


> ## Explanation
> 
> First the table `Employee` is searched to determine the highest salary (= subquery). ​Then the table is searched a second time (= main query) to evaluate each employee's salary against the determined maximum of the subquery. 

# Subquery that returns 1 column 1 row.
## Example 2
Give a list of employees that earn more than the average wage.
1. Determine the average wage
2. Determine the Employees that earn more than this avergae.

In [14]:
USE Xtreme;
SELECT 
 Lastname
,Firstname
,Salary​
FROM Employee​
WHERE Salary > (SELECT AVG(Salary) FROM Employee);

Who (Lastname and Firstname) is the youngest employee from Canada?​
1. What's the birthday of the youngest employee?
2. Whats the birthday of the youngest employee from canada?
3. The actual question: Who (Lastname and Firstname) is the youngest employee from Canada?​ 

In [0]:
USE Xtreme;
SELECT MAX(Birthdate) FROM Employee --WHERE Country = 'Canada')

In [15]:
USE Xtreme;
SELECT 
 Lastname
,Firstname​
FROM Employee
WHERE Country = 'Canada'​
  AND Birthdate = (SELECT MAX(Birthdate) FROM Employee WHERE Country = 'Canada');​

# Subquery that returns 1 column and multiple rows
The result can be used as a list, there the following operators can be used. `IN, NOT IN, ANY, ALL​
> Note that `IN` and `ANY` are equivalent.

# Example 3
1. Give all players(PlayerNo) that played matches.

In [28]:
USE Tennis;
-- Using JOIN From Match to Players
SELECT
 Match.PlayerNo
FROM Matches Match
    JOIN Players Player ON Player.PlayerNo = Match.PlayerNo 

In [27]:
USE Tennis;
-- Using JOIN from Players to Matches
SELECT
 Player.PlayerNo
FROM Players Player
    JOIN Matches Match ON Player.PlayerNo = Match.PlayerNo 

In [20]:
USE Tennis;
-- Using a Subquery
SELECT
 PlayerNo
FROM Players ​
WHERE PlayerNo ANY  (SELECT PlayerNo FROM Matches);

# Example 4
Give name of the players who live in the same town as R. Parmenter​
1. In which column is 'R' located?
2. In which column is 'Parmenter' located?

In [0]:
USE Tennis;
SELECT 
 [Name]
FROM Players​
WHERE Town = (SELECT Town FROM Players WHERE [Name] = 'Parmenter' AND Initials = 'R');

# Example 5
Why is the query below not working?​



In [30]:
USE Xtreme;
SELECT 
 [Name]
FROM Players
WHERE Town = (SELECT Town FROM Players WHERE [Name] = 'Parmenter');

# Example 6
Give all players that **did not play any** matches​

In [33]:
USE Tennis;
SELECT
PlayerNo​
FROM Players​
WHERE PlayerNo NOT IN (SELECT PlayerNo FROM Matches)

In [32]:
USE Tennis;
SELECT
 PlayerNo
FROM Players

EXCEPT

SELECT
PlayerNo
FROM Matches

# `ANY`, `ALL`
Slide 8-9

# Correlated Subqueries


Give employees with a salary larger than the average salary
1. Is this a correlated Subquery?

In [0]:
USE Xtreme;
SELECT 
 Lastname
,Firstname
,Salary​
FROM Employee​
WHERE Salary = (SELECT MAX(Salary)​ FROM Employee)

Give the employees whose salary is larger than the average of the salary of the employees who report to the same boss.​

In [0]:
USE Xtreme;
SELECT 
 Lastname
,Firstname
,Salary​
FROM Employee AS e​
WHERE salary >​ 
    (SELECT AVG(salary)​
     FROM employee​
     WHERE reportsto = e.reportsto);​

# Correlated Subqueries - EXISTS
Give the players that did not play any matches yet.

In [0]:
USE Tennis;
SELECT *​
FROM Players AS p​
WHERE NOT EXISTS​
(SELECT * FROM Matches​ WHERE PlayerNo = p.playerno);​

Give the players that did play matches

In [0]:
USE Tennis;
SELECT *​
FROM Players AS p​
WHERE EXISTS​
(SELECT * FROM Matches​ WHERE PlayerNo = p.playerno);​

#  Multiple ways to achieve the same result:
Who did not play any matches? 
1. Outer Join
2. Simple Subquery
3. Correlated Subquery
4. Except

In [0]:
USE Tennis;
SELECT p.playerno​
FROM players AS p​
     LEFT JOIN​ matches AS m​ ON p.playerno = m.playerno​
WHERE m.playerno IS NULL;

In [0]:
USE Tennis;
SELECT playerno ​
FROM players​
WHERE playerno NOT IN​ (SELECT playerno​ FROM matches);

In [0]:
USE Tennis;
SELECT playerno​
FROM players AS p​
WHERE NOT EXISTS​ (SELECT * FROM matches​ WHERE playerno =  p.playerno);

In [0]:
USE Tennis;
SELECT
 PlayerNo
FROM Players

EXCEPT

SELECT
PlayerNo
FROM Matches

# Derived Tables
Sculpt the table in the from clause, replication of the CASE statement.

In [36]:
USE Xtreme;
SELECT ​
    CASE c.country​
        WHEN 'USA'      THEN 'Northern America'​
        WHEN 'Canada'   THEN 'Northern America'​
        ELSE 'Rest of world' ​
    END AS regionclass, sum(orderamount)​
FROM customer c 
    JOIN orders o ON c.CustomerID = o.CustomerID​
GROUP BY
    CASE c.country​
        WHEN 'USA' then 'Northern America'​
        WHEN 'Canada' then 'Northern America'​
        ELSE 'Rest of world' ​
    END;

(2 rows affected)

Total execution time: 00:00:00.133

regionclass,(No column name)
Rest of world,940289.31
Northern America,3143377.03


In [0]:
USE Xtreme;
SELECT regionclass, sum(total) from​
    (​
    SELECT ​
    CASE c.country​
        WHEN 'USA'      THEN 'Northern America'​
        WHEN 'Canada'   THEN 'Northern America'​
        ELSE 'Rest of world' ​
    END AS region, orderamount​
    FROM customer c 
        JOIN orders o ON c.CustomerID = o.CustomerID​
    ) ​AS totals(regionclass, total)​
GROUP BY regionclass;

Give per productclass the price of the cheapest product and a product that has that price. 

In [0]:
USE Xtreme;
SELECT class, unitprice,​
    SELECT TOP 1 productid​
    WHERE productclassid = class AND price = unitprice​
FROM​
    SELECT productclassid, MIN(price)​
    GROUP BY productclassid​
metaazdata_cell_guid: [],exec

Montly Gross Margin

In [0]:
SELECT 
 ISNULL(Ordered.Month, Purchased.Month) AS [Month]
,ISNULL(Ordered.Amount, 0) - ISNULL(Purchased.Amount, 0) AS [Margin]
FROM
    (
    SELECT 
    FORMAT(OrderDate, 'yyyy-MM')
    ,SUM(OrderAmount)
    FROM Orders
    GROUP BY FORMAT(OrderDate, 'yyyy-MM')
    ) AS Ordered(Month, Amount)
FULL JOIN
    (
    SELECT 
    FORMAT(OrderDate, 'yyyy-MM')
    ,SUM(Product.Price * Purchase.UnitsOnOrder)
    FROM Purchases AS Purchase
        JOIN Product ON Purchase.ProductId = Product.ProductId
    GROUP BY FORMAT(OrderDate, 'yyyy-MM')
    ) AS Purchased(Month, Amount) 
ON Ordered.Month = Purchased.Month
ORDER BY [Month];